#### import libraries 

In [27]:
import os
import cv2
import pandas as pd
import geopandas as gpd
import numpy as np
from collections import Counter

#### file folder ID reading

In [134]:
def read_files():
    list_files = os.listdir('./data/Dang and Eijgenstein/Amsterdam_facade_dataset/train_mask')
    IDs =[]
    for file in list_files: 
        ID_segments = file.split("_")
        # Photo_dist = 
        # print(Photo_dist)
        IDs.append({
            "identificatie": ID_segments[0],
            "photo_dist": ID_segments[1].strip(".png")
        })
    file_df = pd.DataFrame(IDs)

    # save to csv
    IDs_csv_path = './data/list_IDs.csv'
    file_df.to_csv(IDs_csv_path, index=False, header=True)
    return(list_files)
list_files = read_files()
print(list_files)

['0363100012062180_5.220579.png', '0363100012062199_4.616211.png', '0363100012062363_5.239833.png', '0363100012062366_5.126911.png', '0363100012062441_5.056991.png', '0363100012063100_5.555022.png', '0363100012064046_6.976705.png', '0363100012064162_7.164679.png', '0363100012064275_5.676549.png', '0363100012064375_6.006277.png', '0363100012064620_6.643621.png', '0363100012065237_4.700601.png', '0363100012066285_4.44183.png', '0363100012067084_5.457224.png', '0363100012069000_3.982454.png', '0363100012069612_5.446591.png', '0363100012070167_4.467719.png', '0363100012070188_3.356876.png', '0363100012070212_4.540852.png', '0363100012071563_3.988765.png', '0363100012072403_6.650772.png', '0363100012072789_4.491244.png', '0363100012072953_3.889476.png', '0363100012073237_4.813696.png', '0363100012073461_3.983876.png', '0363100012074099_7.400621.png', '0363100012074680_6.482468.png', '0363100012075051_5.165833.png', '0363100012075339_5.74007.png', '0363100012075928_6.952543.png', '0363100012

#### filter BAG by facade mask availability for visualisation in QGIS

In [53]:
def filter_BAG():
    BAG_gdf = gpd.read_file('./data/baglod12citycentre.geojson') # geojson file
    BAG_gdf["identificatie"] = BAG_gdf["identificatie"].str[-15:].astype(str)

    IDs_pdf = pd.read_csv('./data/list_IDs.csv') # CSV file
    IDs_pdf["identificatie"] = IDs_pdf["identificatie"].astype(str)
    joined_gdf = BAG_gdf.merge(IDs_pdf, on="identificatie")

    # Save to geojson
    joined_gdf.to_file("./data/joined_gdf.geojson", driver="GeoJSON")

# filter_BAG()

#### read image(s)

In [48]:
img = cv2.imread("./data/Dang and Eijgenstein/Amsterdam_facade_dataset/train_mask/0363100012179985_4.642188.png")
cv2.imshow("Display", img)
k = cv2.waitKey(0) # Wait for a keystroke in the window

#### read image files, loop

In [32]:
data_dict = []
for file in list_files[:10]: 
    img = cv2.imread(file)
    
    # write the rest of the code to functions
    # execute functions 
    # write cluster info dict, data tree
    # append dict with ID (and photod_dist) to json format
# save data_dict to JSON 

### Extract only the windows from the mask

In [80]:
def retrieve_window_bounds(img):
    
    img_shape= img.shape
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Convert the image to HSV for better color segmentation
    hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    window_fac_val = np.array([102,211,180])
    mask = cv2.inRange(hsv, window_fac_val, window_fac_val)

    #save image
    cv2.imwrite("data/cv2_window_mask.png", mask)
    cv2.imshow("img", mask)
    cv2.waitKey(0)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Approximate contours to polygons + get bounding rects
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)

    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])

    polygon_img = img
    for i in range(len(contours)):
        color = (0,0,255)
        polygon_img = cv2.rectangle(polygon_img, (int(boundRect[i][0]), int(boundRect[i][1])), 
        (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
    # cv2.imwrite("data/cv2_window_contours.png", drawing)
    cv2.imshow('Contours', polygon_img)
    cv2.waitKey(0)

    return boundRect, img_shape

bound_rects, img_shape = retrieve_window_bounds(img)
print(bound_rects)

[(26, 866, 27, 21), (84, 865, 25, 21), (141, 774, 23, 12), (81, 774, 32, 58), (26, 774, 31, 59), (137, 682, 31, 57), (80, 682, 31, 57), (24, 682, 34, 56), (23, 592, 32, 53), (134, 589, 30, 55), (78, 589, 30, 55)]


In [95]:
def retrieve_max_building_height(img): 
    image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    window_fac_val = np.array([0,0,255])

    wall_hsv = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2HSV)
    wall_mask = cv2.inRange(wall_hsv, window_fac_val, window_fac_val)

    wall_contour, _ = cv2.findContours(wall_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Approximate contours to polygons + get bounding rects

    for i in wall_contour:
        wall_contours_poly = cv2.approxPolyDP(i, 3, True)
        wall_boundRect = cv2.boundingRect(wall_contours_poly)
    max_building_height = wall_boundRect[1]
    return max_building_height

max_building_height = retrieve_max_building_height(img)
type(max_building_height)

int

### simplify windows to xy minimum bounding rectangles 

In [65]:
# contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Approximate contours to polygons + get bounding rects
# contours_poly = [None]*len(contours)
# boundRect = [None]*len(contours)

# for i, c in enumerate(contours):
#     contours_poly[i] = cv2.approxPolyDP(c, 3, True)
#     boundRect[i] = cv2.boundingRect(contours_poly[i])
# print(boundRect)                                #  in order of X, Y,(top-left coordinate of the rectangle) and width, height 


# # drawing = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
# # Draw polygonal contour + bonding rects + circles
# polygon_img = img
# for i in range(len(contours)):
#     color = (0,0,255)
#     polygon_img = cv2.rectangle(polygon_img, (int(boundRect[i][0]), int(boundRect[i][1])), 
#     (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
# # cv2.imwrite("data/cv2_window_contours.png", drawing)
# cv2.imshow('Contours', polygon_img)
# k = cv2.waitKey(0) # Wait for a keystroke in the window

[(18, 858, 39, 33), (76, 857, 37, 33), (133, 766, 35, 24), (73, 766, 44, 70), (18, 766, 43, 71), (129, 674, 43, 69), (72, 674, 43, 69), (16, 674, 46, 68), (15, 584, 44, 65), (126, 581, 42, 67), (70, 581, 42, 67)]


### find contours of the building (max building height)

### retrieve additional window information

In [67]:
# #list heights of the upper and lower corner of each window
# y_bounds = []
# window_area = []
# window_height = []
# for border in boundRect:
#     y_bound = []
#     y_bound.append(border[1])

#     upper_corner = border[1] + border[3]
#     y_bound.append(upper_corner)
#     y_bounds.append(y_bound)
#     window_area.append(border[2]*border[3])
#     window_height.append(border[3])
# print(y_bounds)
# print(window_area)
# print(window_height)

[[858, 891], [857, 890], [766, 790], [766, 836], [766, 837], [674, 743], [674, 743], [674, 742], [584, 649], [581, 648], [581, 648]]
[1287, 1221, 840, 3080, 3053, 2967, 2967, 3128, 2860, 2814, 2814]
[33, 33, 24, 70, 71, 69, 69, 68, 65, 67, 67]


### define window clusters

In [131]:
def find_window_clusters(bound_rects): 
    y_bounds = []
    for border in bound_rects:
        y_bound = []
        y_bound.append(border[1])

        upper_corner = border[1] + border[3]
        y_bound.append(upper_corner)
        y_bounds.append(y_bound)
        # window_area.append(border[2]*border[3])
        # window_height.append(border[3])

    window_clusters = []
    for index, window_1 in enumerate(y_bounds):
        new_window = [
            min(window_1[0], min(window_2[0] for window_2 in y_bounds if window_2[0] <= window_1[1] and window_1[0] <= window_2[1])),
            max(window_1[1], max(window_2[1] for window_2 in y_bounds if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]))
        ]
        window_clusters.append(new_window)
            
        counts = Counter(tuple(sublist) for sublist in window_clusters)
        duplicate_indices = {k: [i for i, sublist in enumerate(window_clusters) if tuple(sublist) == k] 
                         for k, _ in counts.items()}
    
    return window_clusters, duplicate_indices

window_clusters, duplicate_indices = find_window_clusters(bound_rects)
print(window_clusters)
print(duplicate_indices)
print(list(duplicate_indices.items()))

[[865, 887], [865, 887], [774, 833], [774, 833], [774, 833], [682, 739], [682, 739], [682, 739], [589, 645], [589, 645], [589, 645]]
{(865, 887): [0, 1], (774, 833): [2, 3, 4], (682, 739): [5, 6, 7], (589, 645): [8, 9, 10]}
[((865, 887), [0, 1]), ((774, 833), [2, 3, 4]), ((682, 739), [5, 6, 7]), ((589, 645), [8, 9, 10])]


### calculate relevant window- & floor information 

In [133]:
def get_cluster_info(bound_rects, duplicate_indices, img_shape, max_building_height):
    window_areas = []
    window_heights = []
    for border in bound_rects:
        window_areas.append(border[2]*border[3])
        window_heights.append(border[3])
    
    cluster_info = []
    floor_level = None
    next_floor_level = None 
    floor_level_save = None
    for i, (_, original_boundary) in enumerate(duplicate_indices.items()):\
    
        #save the roof height from last level as current floor height          
        if floor_level is None: 
            floor_level = img_shape[0]
        else: 
            floor_level = floor_level_save
        win_area_sum = sum(window_areas[i] for i in original_boundary)
        avg_win_height = sum(window_heights[i] for i in original_boundary) / len(original_boundary)

        middle_boundary = (window_clusters[original_boundary[0]][0] + window_clusters[original_boundary[0]][1]) / 2
        avg_sill_height = middle_boundary + avg_win_height / 2
        avg_top_window_height = middle_boundary - avg_win_height/2
        
        if i < len(duplicate_indices) - 1:  # Check if there's a next iteration
            next_cluster = list(duplicate_indices.items())[i + 1][1]
            next_avg_win_height = sum(window_heights[j] for j in next_cluster) / len(next_cluster)
            next_middle_boundary = (window_clusters[next_cluster[0]][0] + window_clusters[next_cluster[0]][1]) / 2
            next_sill = next_middle_boundary + next_avg_win_height / 2
            next_floor_level = next_sill + (2/3*(avg_top_window_height - next_sill))
            floor_level_save = next_floor_level
            floor_height = floor_level-next_floor_level
            wall_area = floor_height * img_shape[1]
            WWR = win_area_sum/wall_area
        else: 
            next_floor_level = max_building_height
            floor_height = floor_level - next_floor_level
            floor_level_save = next_floor_level

            wall_area = floor_height * img_shape[1]
            WWR = win_area_sum/wall_area

        cluster_info.append({
            "window_count": len(original_boundary),
            "indices": original_boundary,
            "boundary": window_clusters[original_boundary[0]],
            "avg_win_height": avg_win_height,
            "floor_level": floor_level,
            "roof_level": next_floor_level,

            "floor_height": floor_height,
            "avg_sill_height": avg_sill_height,
            "window_area_sum": win_area_sum,
            "wall_area": wall_area,
            "WWR": WWR
        })
        
        cluster_df = pd.DataFrame(cluster_info)

    return cluster_df
cluster_info = get_cluster_info(bound_rects, duplicate_indices, img_shape, max_building_height)
print(cluster_info)



   window_count     indices    boundary  avg_win_height  floor_level  \
0             2      [0, 1]  [865, 887]       21.000000   900.000000   
1             3   [2, 3, 4]  [774, 833]       43.000000   852.000000   
2             3   [5, 6, 7]  [682, 739]       56.666667   767.611111   
3             3  [8, 9, 10]  [589, 645]       54.333333   669.500000   

   roof_level  floor_height  avg_sill_height  window_area_sum     wall_area  \
0  852.000000     48.000000       886.500000             1092   9120.000000   
1  767.611111     84.388889       825.000000             3961  16033.888889   
2  669.500000     98.111111       738.833333             5438  18641.111111   
3  466.000000    203.500000       644.166667             4996  38665.000000   

        WWR  
0  0.119737  
1  0.247039  
2  0.291721  
3  0.129212  


In [113]:
def visualise_floors(cluster_info): 
    floors_img = cv2.imread("./data/Dang and Eijgenstein/Amsterdam_facade_dataset/train_mask/0363100012179985_4.642188.png")
    floorimg_shape = floors_img.shape
    # cv2.imshow("Display", img)
    # k = cv2.waitKey(0) # Wait for a keystroke in the window

    #visualise floorlevels 
    for i in range(cluster_info.shape[0]):
        color = (0,0,255)
        cv2.line(floors_img, (0, int(cluster_info.roof_level[i])), #point 1
        (floorimg_shape[1]-1, int(cluster_info.roof_level[i])), #point 2
        color, 2)

    cv2.imshow('floorlevels', floors_img)
    k = cv2.waitKey(0) # Wait for a keystroke in the window
    cv2.imwrite("data/cv2_floor_test.png", floors_img)

visualise_floors(cluster_info)


In [73]:
# def cluster_info_to_m(cluster_info, px_per_m):
#     for i in cluster_info["boundary"][i][0]:
#         i = cluster_info["boundary"][i][0].astype(int)/px_per_m
#     # cluster_info["boundary"][1][0] = cluster_info["boundary"][1][0]/px_per_m
#     return(cluster_info)

# # cluster_info_to_m(cluster_info, 27)
# for i in cluster_info["boundary"]:
#     i = cluster_info["boundary"][i][0].astype(int)/px_per_m
# cluster_info["boundary"][0][0] = cluster_info["boundary"][0][0]/27
# # print(cluster_info["boundary"][0][0])

In [74]:
cluster_info.to_csv("data/cluster_info.csv", sep=";", index=False, header=True)

WHOEEEEEEE yay I think I retrieved all needed information for a facade! 

### my thoughts

In [64]:
# # save CSV to read the color values of the image 
# hsv_2d = hsv.reshape((-1,3))
# df = pd.DataFrame(hsv_2d)
# tiles_csv_path = './data/junk/hsv.csv'
# df.to_csv(tiles_csv_path, index=False, header=False)

# 'not integrated in the code, just read from the file'

# # retrieved color values
# sky_val = np.array([0,0,255])
# cl_fac_val = np.array([60,185,160])
# window_fac_val = np.array([102,211,180])
# door_fac_val = np.array([14,241,255])

# mask = cv2.inRange(hsv, window_fac_val, window_fac_val)

# #save image
# cv2.imwrite("data/cv2_window_mask.png", mask)


# show image 
# cv2.imshow("img", mask)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [75]:
# test, grouping windows 
window_clusters = []
for index, window_1 in enumerate(y_bounds): 
    window_clusters.append(index)
    cluster = []
    for window_2 in y_bounds: 
        if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]: 
            cluster.append(window_2)
    window_clusters.append(cluster)  

print(window_clusters)
print(y_bounds)

[0, [[858, 891], [857, 890]], 1, [[858, 891], [857, 890]], 2, [[766, 790], [766, 836], [766, 837]], 3, [[766, 790], [766, 836], [766, 837]], 4, [[766, 790], [766, 836], [766, 837]], 5, [[674, 743], [674, 743], [674, 742]], 6, [[674, 743], [674, 743], [674, 742]], 7, [[674, 743], [674, 743], [674, 742]], 8, [[584, 649], [581, 648], [581, 648]], 9, [[584, 649], [581, 648], [581, 648]], 10, [[584, 649], [581, 648], [581, 648]]]
[[858, 891], [857, 890], [766, 790], [766, 836], [766, 837], [674, 743], [674, 743], [674, 742], [584, 649], [581, 648], [581, 648]]


In [76]:
last_floor_h = []
# for the lowest average sill height, take difference with 900

for cluster in cluster_info: 
    floor_height = (cluster['floor_height'])
    wall_area = floor_height * img
    WWR = cluster["window_area_sum"]/wall_area
    cluster["WWR"] = WWR
    # for checking if legitimate, devide by 30 to transpose pixels to meters
    print(cluster)


TypeError: string indices must be integers

In [ ]:
# window_clusters = []
# for index, window_1 in enumerate(y_bounds):
#     for window_2 in y_bounds:
#         if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]: 
#             if window_2[0] < window_1[0]:
#                 window_1[0] = window_2[0] 
#             else: 
#                 window_1[0] = window_1[0]
#             if window_2[1] > window_1[1]: 
#                 window_1[1] = window_2[1]
#             else:
#                 window_1[1] = window_1[1] 
#     window_clusters.append(window_1)
# print(window_clusters)
# # count = Counter(tuple(x) for x in window_clusters)
# # print(count)


In [ ]:
# def count_duplicates_with_indices(list_of_lists):
#     counts = Counter(tuple(sublist) for sublist in list_of_lists)
#     duplicates = {k: [i for i, sublist in enumerate(list_of_lists) if tuple(sublist) == k] 
#                   for k, v in counts.items()}
#     return duplicates

# # Example usage (assuming 'y_bounds' is your list of lists):
# duplicate_indices = count_duplicates_with_indices(window_clusters)
# print(duplicate_indices)

# def sum_values_by_duplicate_indices(values, duplicate_indices):
#     sums = {}
#     for duplicate_list in duplicate_indices.values():
#         total = 0
#         for index in duplicate_list:
#             total += values[index]
#         sums[tuple(duplicate_list)] = total
#     return sums

# # Example usage (assuming 'window_area' and 'duplicate_indices' are defined):
# area_sums = sum_values_by_duplicate_indices(window_area, duplicate_indices)
# print(window_area)
# print(area_sums)

# def get_duplicate_info_with_boundaries(boundaries, duplicate_indices):
#     result = []
#     for duplicate_list, area_sum in sum_values_by_duplicate_indices(window_area, duplicate_indices).items():
#         original_boundary = boundaries[duplicate_list[0]]  # Take the first boundary as they are similar
#         result.append({
#             "count": len(duplicate_list),
#             "indices": list(duplicate_list),
#             "boundary": original_boundary,  # Use 'boundary' instead of 'boundaries'
#             "area_sum": area_sum
#         })
#     return result

# # Example usage (assuming 'window_clusters', 'window_area', and 'duplicate_indices' are defined):
# duplicate_info = get_duplicate_info_with_boundaries(window_clusters, duplicate_indices)
# print(duplicate_info)

In [ ]:
# # test, editing the upper and lower margin of each group based on the extremes that are collected
# # trying to do this without the step of first making the sub-lists per window, but instantly adapting the values when finding a next window
# window_clusters = []
# for index, window_1 in enumerate(y_bounds):
#     # low_bound = []
#     # high_bound = []
#     # window_clusters.append(index)
#     # cluster = []
#     for window_2 in y_bounds:
#         if window_2[0] <= window_1[1] and window_1[0] <= window_2[1]: 
#             if window_2[0] < window_1[0]:
#                 window_1[0] = window_2[0] 
#             else: 
#                 window_1[0] = window_1[0]
#             if window_2[1] > window_1[1]: 
#                 window_1[1] = window_2[1]
#                 # high_bound = window_2[1]
#             else:
#                 window_1[1] = window_1[1] 
#                 # high_bound = window_1[1]
#         # print(window_2)
#     # print(window_1) 

#     # print(low_bound)
#     # print(high_bound)
        
#         # high_bound = window_1[1] # high bound is still wrong, trying to wrap my head around it :) 
 
#         # else:
#             # pass
#     # print(low_bound)
#     # print(high_bound)
#             # high_bound = window_2[1]
#             # print(high_bound)
#         #         new_boundary.append(window_2[1])
#         #     else: 
#         #         new_boundary.append(window_1[0])
#         #     print(new_boundary)       
#             # print("yay!")
#             # cluster.append(window_2)
#         # else: 
#             # print("nay..")
#         # if window_1 is in cluster: 
#     window_clusters.append(window_1)
#     # window_clusters.append(low_bound) 
# print(window_clusters)
# # print(window_clusters) 
# # tuple(x) for x in window_clusters:
# #     my_dict = {i:window_clusters.count(i) for i in window_clusters}
# # print(my_dict)
# count = Counter(tuple(x) for x in window_clusters)
# print(count)
# # >>> Counter(tuple(x) for x in lis)
# # Counter({(12, 34, 56): 3, (45, 78, 334): 1, (56, 90, 78): 2})

In [ ]:
# for index, window_1 in enumerate(y_bounds):
#     print(window_1) 
#     window_clusters.append(index)
#     cluster = []
#     for window_2 in y_bounds:
#         high_bound = []
#         print(window_1[0], window_2[1])
#         if window_1[0] <= window_2[1]:
        
#             high_bound = window_2[1]
#             print(high_bound)

todo's